<a href="https://colab.research.google.com/github/Svitlana-py/Forcasting_home_task/blob/main/Lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [1]:
# @title
df = pd.read_csv('house_price.csv')

NameError: name 'pd' is not defined

In [ ]:
df

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df)

In [ ]:
cat_columns = ['Location', 'Has_Garden']

In [ ]:
enc = OrdinalEncoder()
train.loc[:,[col+'_encoded' for col in cat_columns]]=enc.fit_transform(train[cat_columns])
test.loc[:,[col+'_encoded' for col in cat_columns]]=enc.transform(test[cat_columns])

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
target = 'House_Price'
features = list(df.columns.drop([target]+cat_columns))+[col+'_encoded' for col in cat_columns]
features

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

train[target]

In [ ]:
mod = DecisionTreeRegressor (max_depth=2)
mod.fit(train[features], train[target])
preds = mod.predict(test[features])
r2_score(test[target], preds)

In [ ]:
mean_House_Price = test.House_Price.mean()
print("{:.2f}".format(mean_House_Price))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer

In [ ]:
col_transformer = ColumnTransformer(transformers=[('ord_encoder', OrdinalEncoder(), cat_columns)],
                                   remainder='passthrough',
                                   force_int_remainder_cols=False)
pipe = Pipeline([('column_transformer', col_transformer), ('estimator', mod)])

In [ ]:
scoring_metrics = make_scorer(r2_score,
                              greater_is_better=True
                             )

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(pipe, df.drop(target, axis=1), df[target], cv = 6,
                         scoring=scoring_metrics
                         )

In [ ]:
scores

---
***Grid search***
---







In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
x,y = df.drop(target, axis=1), df[target]
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [ ]:
param_grid = {'estimator__max_depth': range(2, 10)}

In [ ]:
grid_search = GridSearchCV(pipe,
                          param_grid,
                          cv=5,
                          verbose=True, refit=True)

In [ ]:
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
r2_score(y_test,grid_search.best_estimator_.predict(x_test))